In [1]:
import random
import dbutils
from datetime import date
from time import mktime
import calendar
import numpy as np
import glob
import os
from keras import models
import progressbar
import sys
sys.path.append("..")
from cgmcore import modelutils
from cgmcore import utils

main_connector = dbutils.connect_to_main_database()

/data/anaconda/envs/datathon/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Using TensorFlow backend.


# Preparing the PCD data.

In [2]:
# This is the main variable.
pcd_megabatches = []

# The table to query.
data_table = "pointcloud_data"

# The values of the measurents-table that should appear in the results.
values = ["height_cms"]
values = ["measurements." + value for value in values]
values = ", ".join(values)


for year in [2018, 2019]:
    for month in range(1, 13):
        
        # Getting the start.
        start_day = 1
        start_date = date(year, month, start_day)
        start_timestamp = str(1000 * int(mktime(start_date.timetuple())))
        
        # Getting the end.
        _, end_day = calendar.monthrange(year, month)
        end_date = date(year, month, end_day)
        end_timestamp = str(1000 * int(mktime(end_date.timetuple())))

        # Compose an SQL-statement.
        sql_statement = ""
        sql_statement += "SELECT {}.path, {}".format(data_table, values)
        sql_statement += " FROM {}".format(data_table)
        sql_statement += " INNER JOIN measurements ON {}.measurement_id=measurements.id".format(data_table)
        sql_statement += " WHERE measurements.type=\'manual\'"
        sql_statement += " AND measurements.timestamp >= {}".format(start_timestamp)
        sql_statement += " AND measurements.timestamp <= {}".format(end_timestamp)
        sql_statement += " AND timestamp <= {}".format(end_timestamp)
        results = main_connector.execute(sql_statement, fetch_all=True)
        if len(results) != 0:
            pcd_megabatches.append((f"{year}-{month}", results))

all_results = []
for _, results in pcd_megabatches:
    all_results.extend(results)
pcd_megabatches.append(("all-data", all_results))

    
for tag, results in pcd_megabatches:
    print("Tag {} has {} PCDs.".format(tag, len(results)))


print("")
print("{} megabatches to check.".format(len(pcd_megabatches)))

Tag 2018-9 has 2063 PCDs.
Tag 2018-10 has 1871 PCDs.
Tag 2018-11 has 385 PCDs.
Tag 2018-12 has 443 PCDs.
Tag all-data has 4762 PCDs.

5 megabatches to check.


# Preparing models.

In [3]:
# Getting all the model paths.
models_root_path = "/whhdata/models"
model_paths = sorted(glob.glob(os.path.join(models_root_path, "*.h5")))
model_paths = [model_path for model_path in model_paths if model_path.endswith("-model-weights.h5") == False]
print("Found {} models".format(len(model_paths)))

# Load one spare point net model.
point_net_model = modelutils.create_point_net((30000, 3), 1, hidden_sizes = [512, 256])

Found 16 models


# Evaluating models against mega-batches.

In [4]:
#batch_size = 128

model_load_errors = 0

# Routine for loading models.
def load_model(model_path):
    # Try to load the model.
    try:
        model = models.load_model(model_path)
    except ValueError:
        # Error. Could be a problem with pointnet, which needs to be loaded via weights.
        try:
            model = point_net_model
            model.load_weights(model_path.replace("-model.h5", "-model-weights.h5"))
        except Exception as e:
            # All hope is lost.
            print(e)
            print("Error loading model.")
            return None
    return model
 
# Evaluating models.
def evaluate_model(model):
    evaluation_results = []
    
    input_shape = model.inputs[0].shape[1:]
    model_type = ""
    if len(input_shape) == 3 and input_shape == (32, 32, 32):
        model_type ="voxnet"
    elif len(input_shape) == 3 and input_shape[2] == 3:
        model_type ="rgb"
    elif len(input_shape) == 2 and input_shape == (30000, 3):
        model_type ="pointcloud"
    else:
        print("Unexpected:", input_shape)
        
    output_shape = model.outputs[0].shape[1:]
    print(input_shape, output_shape)
    
    for tag, samples in pcd_megabatches:
        print("{}: {} PCDs.".format(tag, len(samples)))
        
        x_input = []
        y_output = []
        bar = progressbar.ProgressBar(max_value=len(samples))
        for index, (path, target) in enumerate(samples):
            bar.update(index)
            try:
                pcd_array = utils.load_pcd_as_ndarray(path)
                rgb_map = utils.pointcloud_to_rgb_map(pcd_array, target_width=64, target_height=64, scale_factor=1.0, axis="vertical")
                x_input.append(rgb_map)
                y_output.append(target)
            except Exception as e:
                print(e)
        bar.finish()  
        
        x_input = np.array(x_input)
        y_output = np.array(y_output)

        mse, mae = model.evaluate(x_input, y_output)
        print(tag, mse, mae)
        evaluation_results.append((tag, mse, mae))
    return evaluation_results
        
        
# Loop through all models.
for model_index, model_path in enumerate(model_paths):
    print(f"Evaluating{model_path}...")
    
    # Skip voxnets and pointnets (debugging).
    if "voxnet" in model_path or "pointnet" in model_path:
        print("Skipped.")
        continue
    
    # Attempt to load the model.
    model = load_model(model_path)
    if model == None:
        model_load_errors += 1
        continue
    
    # Evaluate the model.
    # TODO put this to a database.
    evaluation_results = evaluate_model(model)
    print(f"Results for {model_path}:")
    for tag, mse, mae in evaluation_results:
        print(f"{tag} mse: {mse} mae: {mae}")

    del model
    print("")
    
    
print(f"{model_load_errors}/{len(model_paths)} models could not be loaded due to errors.")
    #print("Evaluating model \"{}\"...".format(model_path))
    #for batch_index, index in enumerate(range(0, len(results), batch_size)):
    #    print(batch_index, index, index + batch_size)

Evaluating/whhdata/models/20181026-0710-voxnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181029-1312-voxnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181029-1314-voxnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181029-1801-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181031-1201-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181031-1636-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181031-2038-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181101-1444-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181101-1643-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181101-2145-pointnet-model.h5...
Skipped.
Evaluating/whhdata/models/20181104-0039-2dCNN-model.h5...


N/A% (0 of 2063) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

(64, 64, 3) (1,)
2018-9: 2063 PCDs.


 98% (2025 of 2063) |################### | Elapsed Time: 0:16:00 ETA:   0:00:13

cannot reshape array of size 0 into shape (0,newaxis)


...

(256, 256, 3) (1,)
2018-9: 2063 PCDs.


 98% (2025 of 2063) |################### | Elapsed Time: 0:16:17 ETA:   0:00:13

cannot reshape array of size 0 into shape (0,newaxis)


100% (2063 of 2063) |####################| Elapsed Time: 0:16:35 Time:  0:16:35


ValueError: Error when checking input: expected vgg19_input to have shape (256, 256, 3) but got array with shape (64, 64, 3)